In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
import seaborn as sns
plt.rc('font', family='NanumBarunGothic') # '나눔고딕' 폰트 적용



In [16]:
train = pd.read_csv('병원train.csv')
test = pd.read_csv('병원test.csv')

train.shape, test.shape

((301, 58), (127, 58))

In [3]:
# train.head()

In [3]:
train.columns

Index(['inst_id', 'OC', 'sido', 'sgg', 'openDate', 'bedCount', 'instkind',
       'revenue1', 'salescost1', 'sga1', 'salary1', 'noi1', 'noe1',
       'interest1', 'ctax1', 'profit1', 'liquidAsset1', 'quickAsset1',
       'receivableS1', 'inventoryAsset1', 'nonCAsset1', 'tanAsset1',
       'OnonCAsset1', 'receivableL1', 'debt1', 'liquidLiabilities1',
       'shortLoan1', 'NCLiabilities1', 'longLoan1', 'netAsset1', 'surplus1',
       'revenue2', 'salescost2', 'sga2', 'salary2', 'noi2', 'noe2',
       'interest2', 'ctax2', 'profit2', 'liquidAsset2', 'quickAsset2',
       'receivableS2', 'inventoryAsset2', 'nonCAsset2', 'tanAsset2',
       'OnonCAsset2', 'receivableL2', 'debt2', 'liquidLiabilities2',
       'shortLoan2', 'NCLiabilities2', 'longLoan2', 'netAsset2', 'surplus2',
       'employee1', 'employee2', 'ownerChange'],
      dtype='object')

In [4]:
train_column=pd.DataFrame({'columns':train.columns,
                        'info' :['각 파일에서의 병원 고유 번호',
                                      '영업/폐업 분류, 2018년 폐업은 2017년 폐업으로 간주함',
                                      '병원의 광역 지역 정보',
                                      '병원의 시군구 자료',
                                      '병원 설립일','병원이 갖추고 있는 병상의 수',
                                      '병원, 의원, 요양병원, 한의원, 종합병원 등 병원의 종류 종합병원 : 입원환자 100명 이상 수용 가능 병원 : 입원 환자 30명 이상 100명 미만 수용 가능 의원 : 입원 환자 30명 이하 수용 가능 한방 병원(한의원) : 침술과 한약으로 치료하는 의료 기관.',
                                      '매출액, 2017(회계년도)년 데이터를 의미함',
                                      '매출원가, 2017(회계년도)년 데이터를 의미함',
                                      '판매비와 관리비, 2017(회계년도)년 데이터를 의미함',
                                      '급여, 2017(회계년도)년 데이터를 의미함','영업외수익, 2017(회계년도)년 데이터를 의미함',
                                      '영업외비용, 2017(회계년도)년 데이터를 의미함','이자비용, 2017(회계년도)년 데이터를 의미함',
                                      '법인세비용, 2017(회계년도)년 데이터를 의미함','당기순이익, 2017(회계년도)년 데이터를 의미함',
                                      '유동자산, 2017(회계년도)년 데이터를 의미함','당좌자산, 2017(회계년도)년 데이터를 의미함',
                                      '미수금(단기), 2017(회계년도)년 데이터를 의미함','재고자산, 2017(회계년도)년 데이터를 의미함',
                                      '비유동자산, 2017(회계년도)년 데이터를 의미함','유형자산, 2017(회계년도)년 데이터를 의미함',
                                      '기타 비유동자산, 2017(회계년도)년 데이터를 의미함','장기미수금, 2017(회계년도)년 데이터를 의미함',
                                      '부채총계, 2017(회계년도)년 데이터를 의미함','유동부채, 2017(회계년도)년 데이터를 의미함',
                                      '단기차입금, 2017(회계년도)년 데이터를 의미함','비유동부채, 2017(회계년도)년 데이터를 의미함',
                                      '장기차입금, 2017(회계년도)년 데이터를 의미함','순자산총계, 2017(회계년도)년 데이터를 의미함','이익잉여금, 2017(회계년도)년 데이터를 의미함',
                                      '매출액, 2016(회계년도)년 데이터를 의미함','매출원가, 2016(회계년도)년 데이터를 의미함',
                                      '판매비와 관리비, 2016(회계년도)년 데이터를 의미함','급여, 2016(회계년도)년 데이터를 의미함',
                                      '영업외수익, 2016(회계년도)년 데이터를 의미함','영업외비용, 2016(회계년도)년 데이터를 의미함',
                                      '이자비용, 2016(회계년도)년 데이터를 의미함','법인세비용, 2016(회계년도)년 데이터를 의미함',
                                      '당기순이익, 2016(회계년도)년 데이터를 의미함','유동자산, 2016(회계년도)년 데이터를 의미함',
                                      '당좌자산, 2016(회계년도)년 데이터를 의미함','미수금(단기), 2016(회계년도)년 데이터를 의미함',
                                      '재고자산, 2016(회계년도)년 데이터를 의미함','비유동자산, 2016(회계년도)년 데이터를 의미함',
                                      '유형자산, 2016(회계년도)년 데이터를 의미함','기타 비유동자산, 2016(회계년도)년 데이터를 의미함',
                                      '장기미수금, 2016(회계년도)년 데이터를 의미함','부채총계, 2016(회계년도)년 데이터를 의미함',
                                      '유동부채, 2016(회계년도)년 데이터를 의미함','단기차입금, 2016(회계년도)년 데이터를 의미함',
                                      '비유동부채, 2016(회계년도)년 데이터를 의미함','장기차입금, 2016(회계년도)년 데이터를 의미함',
                                      '순자산총계, 2016(회계년도)년 데이터를 의미함','이익잉여금, 2016(회계년도)년 데이터를 의미함',
                                      '고용한 총 직원의 수, 2017(회계년도)년 데이터를 의미함','고용한 총 직원의 수, 2016(회계년도)년 데이터를 의미함',
                                      '대표자의 변동 여부']})
                                      


In [5]:
display(train_column)

,columns,info
0,inst_id,각 파일에서의 병원 고유 번호
1,OC,"영업/폐업 분류, 2018년 폐업은 2017년 폐업으로 간주함"
2,sido,병원의 광역 지역 정보
3,sgg,병원의 시군구 자료
4,openDate,병원 설립일
5,bedCount,병원이 갖추고 있는 병상의 수
6,instkind,"병원, 의원, 요양병원, 한의원, 종합병원 등 병원의 종류 종합병원 : 입원환자 1..."
7,revenue1,"매출액, 2017(회계년도)년 데이터를 의미함"
8,salescost1,"매출원가, 2017(회계년도)년 데이터를 의미함"
9,sga1,"판매비와 관리비, 2017(회계년도)년 데이터를 의미함"


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 58 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   inst_id             301 non-null    int64  
 1   OC                  301 non-null    object 
 2   sido                301 non-null    object 
 3   sgg                 301 non-null    int64  
 4   openDate            301 non-null    int64  
 5   bedCount            296 non-null    float64
 6   instkind            300 non-null    object 
 7   revenue1            293 non-null    float64
 8   salescost1          293 non-null    float64
 9   sga1                293 non-null    float64
 10  salary1             293 non-null    float64
 11  noi1                293 non-null    float64
 12  noe1                293 non-null    float64
 13  interest1           293 non-null    float64
 14  ctax1               293 non-null    float64
 15  profit1             293 non-null    float64
 16  liquidAs

In [17]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 58 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   inst_id             127 non-null    int64  
 1   OC                  0 non-null      float64
 2   sido                127 non-null    object 
 3   sgg                 127 non-null    int64  
 4   openDate            126 non-null    float64
 5   bedCount            119 non-null    float64
 6   instkind            125 non-null    object 
 7   revenue1            125 non-null    float64
 8   salescost1          125 non-null    float64
 9   sga1                125 non-null    float64
 10  salary1             125 non-null    float64
 11  noi1                125 non-null    float64
 12  noe1                125 non-null    float64
 13  interest1           125 non-null    float64
 14  ctax1               125 non-null    float64
 15  profit1             125 non-null    float64
 16  liquidAs

## #01.데이터 전처리

### OC 병원 개폐업 구분

OC는 종속변수기 때문에 정상임

In [11]:
train['OC'].unique()

array(['open', ' close'], dtype=object)

In [18]:
test['OC'].unique()

array([nan])

### openDate

In [19]:
train['openDate'].dtypes

dtype('int64')

In [20]:
test['openDate'].dtypes

dtype('float64')

In [23]:
train['openDate'].isnull().sum()

0

In [22]:
test['openDate'].isnull().sum()

1

In [25]:
test['openDate']=  test['openDate'].fillna(-1)
test['openDate'] =test['openDate'].astype('int')
test['openDate'].dtypes 

dtype('int32')

### employess 1,2

In [26]:
print('train dtype : ',train['employee1'].dtypes)
print('test dtype : ',test['employee1'].dtypes)

train dtype :  float64
test dtype :  object


In [27]:
print('train dtype : ',train['employee2'].dtypes)
print('test dtype : ',test['employee2'].dtypes)

train dtype :  float64
test dtype :  object


In [29]:
test['employee1'].unique()

array(['693', '379', nan, '760', '437', '386', '727', '377', '17', '532',
       '128', '1,637', '1,816', '566', '104', '244', '187', '431', '150',
       '361', '41', '110', '457', '210', '344', '250', '48', '109', '292',
       '121', '83', '100', '154', '134', '107', '89', '86', '69', '93',
       '324', '1,065', '340', '55', '22', '300', '120', '260', '143',
       '78', '81', '46', '74', '155', '130', '122', '59', '180', '855',
       '30', '96', '75', '44', '203', '70', '129', '67', '85', '236',
       '29', '116', '37', '25', '45', '47', '52', '56', '76', '125', '80',
       '63', '90', '190', '222', '32', '66', '224', '61', '28', '604',
       '33', '16', '36', '106', '60', '19', '49', '21', '111', '560',
       '132', '363'], dtype=object)

In [31]:
test['employee1'] = test['employee1'].str.replace(',','')
test['employee2'] = test['employee2'].str.replace(',','')

test['employee1'] = test['employee1'].astype('float')
test['employee2'] = test['employee2'].astype('float')



AttributeError: Can only use .str accessor with string values!

In [32]:
print('employee1 : ',test['employee1'].dtypes)
print('employee2 : ',test['employee2'].dtypes)

employee1 :  float64
employee2 :  float64


### 기초통계량 확인

#### 수치형

In [33]:
display(train.describe())


,inst_id,sgg,openDate,bedCount,revenue1,salescost1,sga1,salary1,noi1,noe1,...,receivableL2,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2
count,301.000000,301.000000,3.010000e+02,296.000000,2.930000e+02,2.930000e+02,2.930000e+02,2.930000e+02,2.930000e+02,2.930000e+02,...,2.930000e+02,2.930000e+02,2.930000e+02,2.930000e+02,2.930000e+02,2.930000e+02,2.930000e+02,2.930000e+02,291.000000,288.000000
mean,219.056478,81.039867,2.005013e+07,145.709459,1.288175e+10,2.014903e+09,1.033244e+10,5.654115e+09,2.696151e+08,5.115870e+08,...,2.226700e+05,8.146026e+09,3.860584e+09,1.510050e+09,4.471247e+09,2.709979e+09,5.273919e+09,9.786279e+08,142.546392,134.326389
std,121.234869,50.969714,8.893815e+04,118.923890,2.043543e+10,7.460271e+09,1.493886e+10,8.083343e+09,8.023701e+08,1.060379e+09,...,3.811496e+06,1.255800e+10,6.797242e+09,2.953412e+09,7.659580e+09,4.564001e+09,1.081259e+10,4.688798e+09,160.191073,151.061786
min,1.000000,1.000000,1.978012e+07,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.014780e+10,-2.781507e+09,0.000000,0.000000
25%,112.000000,37.000000,2.001102e+07,52.750000,3.252112e+09,0.000000e+00,2.758201e+09,1.626053e+09,8.217133e+06,8.013395e+07,...,0.000000e+00,1.283220e+09,2.855741e+08,0.000000e+00,2.557878e+07,0.000000e+00,1.017573e+09,0.000000e+00,53.500000,53.750000
50%,230.000000,75.000000,2.007113e+07,136.500000,5.524218e+09,2.104105e+08,4.684074e+09,2.659892e+09,4.363764e+07,1.831965e+08,...,0.000000e+00,3.784553e+09,1.454050e+09,8.542549e+07,1.969747e+09,1.100000e+09,2.894970e+09,0.000000e+00,80.000000,79.000000
75%,321.000000,123.000000,2.011102e+07,193.000000,1.274839e+10,9.105278e+08,1.077876e+10,6.363400e+09,2.050331e+08,4.203330e+08,...,0.000000e+00,8.465053e+09,4.364714e+09,1.567967e+09,4.905441e+09,3.360000e+09,5.370285e+09,1.014447e+08,181.500000,170.000000
max,428.000000,178.000000,2.017061e+07,656.000000,1.510000e+11,9.850332e+10,1.030000e+11,6.403559e+10,9.144171e+09,8.686380e+09,...,6.524231e+07,8.508858e+10,6.846878e+10,1.759375e+10,5.150388e+10,3.256147e+10,1.250000e+11,6.852730e+10,1200.000000,1200.000000


#### 범주형

In [34]:
train.describe(include='object')

,OC,sido,instkind,ownerChange
count,301,301,300,289
unique,2,16,7,2
top,open,gyeonggi,nursing_hospital,same
freq,286,49,144,247


In [38]:
cat_col = ['OC',	'sido',	'instkind',	'ownerChange']
num_col = train.describe().columns


In [39]:
print('--------------- train 수치형변수 범위 --------------')
for col in num_col :
    print(col,': ',train[col].min(),'~',train[col].max())

--------------- train 수치형변수 범위 --------------
inst_id :  1 ~ 428
sgg :  1 ~ 178
openDate :  19780124 ~ 20170607
bedCount :  0.0 ~ 656.0
revenue1 :  0.0 ~ 151000000000.0
salescost1 :  0.0 ~ 98503322990.0
sga1 :  0.0 ~ 103000000000.0
salary1 :  0.0 ~ 64035593950.0
noi1 :  0.0 ~ 9144170639.0
noe1 :  0.0 ~ 8686379500.0
interest1 :  0.0 ~ 2841474870.0
ctax1 :  0.0 ~ 2808883191.0
profit1 :  -4696700774.0 ~ 7475426881.0
liquidAsset1 :  0.0 ~ 42808356695.0
quickAsset1 :  0.0 ~ 41791456468.0
receivableS1 :  0.0 ~ 14092571696.0
inventoryAsset1 :  0.0 ~ 2067763564.0
nonCAsset1 :  0.0 ~ 156000000000.0
tanAsset1 :  0.0 ~ 156000000000.0
OnonCAsset1 :  0.0 ~ 10085168680.0
receivableL1 :  0.0 ~ 73742310.0
debt1 :  0.0 ~ 86697249913.0
liquidLiabilities1 :  0.0 ~ 39373879374.0
shortLoan1 :  0.0 ~ 19885197610.0
NCLiabilities1 :  0.0 ~ 55017362619.0
longLoan1 :  0.0 ~ 41124897062.0
netAsset1 :  -20196244362.0 ~ 139000000000.0
surplus1 :  -2914969563.0 ~ 134000000000.0
revenue2 :  0.0 ~ 137000000000.0
sale

In [40]:
print('--------------- test 수치형변수 범위 --------------')
for col in num_col :
    print(col,': ',test[col].min(),'~',test[col].max())

--------------- test 수치형변수 범위 --------------
inst_id :  2 ~ 431
sgg :  6 ~ 177
openDate :  -1 ~ 20170531
bedCount :  0.0 ~ 771.0
revenue1 :  0.0 ~ 181000000000.0
salescost1 :  0.0 ~ 116000000000.0
sga1 :  0.0 ~ 127000000000.0
salary1 :  0.0 ~ 82395611833.0
noi1 :  0.0 ~ 7302719335.0
noe1 :  0.0 ~ 11496795211.0
interest1 :  0.0 ~ 2573803974.0
ctax1 :  0.0 ~ 2308489420.0
profit1 :  -5734451117.0 ~ 11208159317.0
liquidAsset1 :  0.0 ~ 43870284311.0
quickAsset1 :  0.0 ~ 41979652806.0
receivableS1 :  0.0 ~ 21441826107.0
inventoryAsset1 :  0.0 ~ 1818617919.0
nonCAsset1 :  0.0 ~ 130000000000.0
tanAsset1 :  0.0 ~ 117000000000.0
OnonCAsset1 :  0.0 ~ 7537348637.0
receivableL1 :  0.0 ~ 325392770.0
debt1 :  0.0 ~ 129000000000.0
liquidLiabilities1 :  0.0 ~ 68916350701.0
shortLoan1 :  0.0 ~ 38059269376.0
NCLiabilities1 :  0.0 ~ 85286333620.0
longLoan1 :  0.0 ~ 26425324332.0
netAsset1 :  -3821589613.0 ~ 49434037769.0
surplus1 :  -8636717673.0 ~ 34114713245.0
revenue2 :  0.0 ~ 170000000000.0
salescost2

In [43]:
print('--------------- train 수치형변수 범위 --------------')
for col in cat_col :
    print(col,': ',train[col].unique())

--------------- train 수치형변수 범위 --------------
OC :  ['open' ' close']
sido :  ['choongnam' 'gyeongnam' 'gyeonggi' 'incheon' 'busan' 'jeonnam' 'seoul'
 'jeonbuk' 'choongbuk' 'ulsan' 'daejeon' 'daegu' 'gyeongbuk' 'gangwon'
 'gwangju' 'sejong']
instkind :  ['nursing_hospital' 'general_hospital' 'hospital' 'traditional_clinic'
 'clinic' 'traditional_hospital' 'dental_clinic' nan]
ownerChange :  ['same' 'change' nan]


In [45]:
print('--------------- test 수치형변수 범위 --------------')
for col in cat_col :
    print(col,': ',test[col].unique())

--------------- test 수치형변수 범위 --------------
OC :  [nan]
sido :  ['incheon' 'jeju' 'jeonnam' 'busan' 'jeonbuk' 'gyeonggi' 'gyeongbuk'
 'daejeon' 'seoul' 'daegu' 'choongbuk' 'ulsan' 'gyeongnam' 'choongnam'
 'sejong']
instkind :  ['general_hospital' 'hospital' 'nursing_hospital' 'traditional_clinic'
 'clinic' nan 'traditional_hospital']
ownerChange :  ['same' nan 'change']
